In [1]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-892637ff-5e5d-4f51-a930-35a14752a855',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': '7KVA3B7Hwls50CZb1wjKSGRMNmlS_q0k4Aij2BxWDKz-'
}

configuration_name = 'os_bc8d67bce2464886a631a0fb8e956747_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet(cos.url('hmp.parquet', 'first-donotdelete-pr-j0c5ggpr4rlsce'))
df.show()


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191228125520-0000
KERNEL_ID = 26e96a04-ca4e-4583-a0be-5fe650d9c329
+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth

In [2]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['x','y','z'], outputCol ='features')

In [3]:
from pyspark.ml.clustering import KMeans
km = KMeans().setK(13).setSeed(1)

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler, km])
model = pipeline.fit(df)
wsse = model.stages[1].computeCost(assembler.trasform(df))
wsse